In [2]:
import requests
import json
from collections import defaultdict

# GitHub repository information
owner = 'lucamenotti'
repo = '5v5s'
branch = 'main'  # Or any other branch you want to fetch from

# GitHub API endpoint to list files in the repository
api_url = f'https://api.github.com/repos/{owner}/{repo}/contents'

# Alias mapping (key: alias, value: primary name)
player_aliases = {
    'EnderDragon7474': 'Ashalo1',
    'Ashalo1': 'Ashalo1',
    'RaVe B00ST': 'Bryant',
    'CoomerHashira': 'Bryant',
    'Item Diversity': 'BloodStainedRose',
    'BigPeTe1239': 'DookieDemon12'
    # Add more mappings as needed
}

def get_primary_name(name):
    """Returns the primary name for a given player name using the alias mapping."""
    return player_aliases.get(name, name)  # Return primary name if alias exists, otherwise the name itself

def fetch_files_from_github(api_url, path=''):
    """Fetches JSON files from the given GitHub repository path."""
    files = []
    response = requests.get(f'{api_url}/{path}')
    if response.status_code == 200:
        for item in response.json():
            if item['type'] == 'file' and item['name'].endswith('.json'):
                files.append(item['download_url'])
            elif item['type'] == 'dir':
                files.extend(fetch_files_from_github(api_url, item['path']))
    else:
        print(f"Failed to fetch files: {response.status_code} - {response.text}")
    return files

def fetch_json_from_url(url):
    """Fetches JSON data from the given URL."""
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch JSON from {url}: {response.status_code} - {response.text}")
    return None

# Fetch JSON files from the GitHub repository
json_files_urls = fetch_files_from_github(api_url)

# Initialize player stats dictionary
player_stats = defaultdict(lambda: {
    'wins': 0,
    'losses': 0,
    'kills': 0,
    'deaths': 0,
    'assists': 0,
    'matches': 0,
    'total_kills': 0,
    'penta_kills': 0,
    'games_played': 0
})

# Initialize teammate tracking dictionary for same team
teammate_counts = defaultdict(lambda: defaultdict(int))

# Parse and extract data from each JSON file
for file_url in json_files_urls:
    match = fetch_json_from_url(file_url)
    if match:
        if 'participants' not in match:
            print(f"No participants data in file: {file_url}")
            continue
        participants = match['participants']
        # Group players by team
        teams = defaultdict(list)
        for player in participants:
            if 'NAME' not in player or 'TEAM' not in player:
                print(f"Incomplete player data in file: {file_url}")
                continue
            primary_name = get_primary_name(player['NAME'])
            team_id = player['TEAM']
            teams[team_id].append(primary_name)
        
        # Update player stats and teammate counts
        for player in participants:
            if 'NAME' not in player or 'TEAM' not in player:
                continue
            player_name = get_primary_name(player['NAME'])  # Use primary name
            player_stats[player_name]['matches'] += 1
            player_stats[player_name]['kills'] += int(player.get('CHAMPIONS_KILLED', 0))
            player_stats[player_name]['assists'] += int(player.get('ASSISTS', 0))
            player_stats[player_name]['deaths'] += int(player.get('NUM_DEATHS', 0))
            player_stats[player_name]['total_kills'] += int(player.get('CHAMPIONS_KILLED', 0))
            player_stats[player_name]['penta_kills'] += int(player.get('PENTA_KILLS', 0))
            player_stats[player_name]['games_played'] += 1

            # Check win/loss
            if player.get('WIN') == 'Win':
                player_stats[player_name]['wins'] += 1
            else:
                player_stats[player_name]['losses'] += 1
            
            # Update teammate counts for players on the same team
            team_id = player['TEAM']
            teammates = teams[team_id]
            for teammate in teammates:
                if teammate != player_name:  # Exclude the player themselves
                    teammate_counts[player_name][teammate] += 1

if not player_stats:
    print("No player data found.")
else:
    # Compute win rate and average kills per match
    for player_name, stats in player_stats.items():
        stats['win_rate'] = stats['wins'] / stats['matches'] * 100
        stats['avg_kills_per_match'] = stats['kills'] / stats['matches']

    # Sort players by the number of matches played, descending
    sorted_players = sorted(player_stats.items(), key=lambda x: x[1]['matches'], reverse=True)
    
    # Find player with the most deaths
    most_deaths_player = max(player_stats.items(), key=lambda x: x[1]['deaths'])
    most_kills_player = max(player_stats.items(), key=lambda x: x[1]['kills'])
    print(f"Player with the most deaths: {most_deaths_player[0]} with {most_deaths_player[1]['deaths']} deaths")
    print(f"Player with the most kills: {most_kills_player[0]} with {most_kills_player[1]['kills']} kills")
    
    # Open stats file
    z = open("stats.txt", "w")
    
    # Print statistics for each player
    for player_name, stats in sorted_players:
        z.write(f"Player: {player_name}")
        print(f"Player: {player_name}")
        z.write(f"  Win Rate: {stats['win_rate']:.2f}%")
        print(f"  Win Rate: {stats['win_rate']:.2f}%")
        z.write(f"  Games Played: {stats['games_played']}")
        print(f"  Games Played: {stats['games_played']}")
        z.write(f"  Average Kills per Match: {stats['avg_kills_per_match']:.2f}")
        print(f"  Average Kills per Match: {stats['avg_kills_per_match']:.2f}")
        avg_kda = (stats['kills'] + stats['assists']) / (stats['deaths'] if stats['deaths'] != 0 else 1)
        avg_kda = round(avg_kda, 2)
        z.write(f"  Average KDA: {avg_kda}\n")
        print(f"  Average KDA: {avg_kda}\n")
    z.close()

Player with the most deaths: IbelieveMeYou with 302 deaths
Player with the most kills: adostraa with 427 kills
Player: adostraa
  Win Rate: 58.33%
  Games Played: 60
  Average Kills per Match: 7.12
  Average KDA: 3.53

Player: nigeriacoc
  Win Rate: 52.54%
  Games Played: 59
  Average Kills per Match: 5.03
  Average KDA: 2.97

Player: TylerToTheT212
  Win Rate: 67.86%
  Games Played: 56
  Average Kills per Match: 7.32
  Average KDA: 3.91

Player: IbelieveMeYou
  Win Rate: 48.15%
  Games Played: 54
  Average Kills per Match: 6.74
  Average KDA: 2.56

Player: Ashalo1
  Win Rate: 39.58%
  Games Played: 48
  Average Kills per Match: 6.40
  Average KDA: 2.59

Player: ihyash
  Win Rate: 53.85%
  Games Played: 39
  Average Kills per Match: 2.97
  Average KDA: 4.02

Player: DonutDude17
  Win Rate: 63.16%
  Games Played: 38
  Average Kills per Match: 3.50
  Average KDA: 2.57

Player: GasterBat
  Win Rate: 68.75%
  Games Played: 32
  Average Kills per Match: 4.03
  Average KDA: 2.31

Player: Bry

In [3]:
    # Function to get the number of games played with each teammate on the same team
    def get_teammates_count(player_name):
        primary_name = get_primary_name(player_name)
        return teammate_counts[primary_name]
    
    # Function to calculate average winrate of average teammates
    for player_name in sorted_players:
        player_name = player_name[0]
        total_count=0
        total_win_rate=0
        teammates_count = get_teammates_count(player_name)
        #print(f"Teammates count for player {player_name}:")
        for teammate, count in teammates_count.items():
            current_win_rate = player_stats[teammate]['win_rate']
            total_win_rate += current_win_rate*count
            total_count += count
            #print(f"  Teammate: {teammate}, Games Played Together: {count}, Win Rate:{current_win_rate} Total Win Rate: {total_win_rate}, Total Count: {total_count}")

        print(f"{player_name}'s teammates vs self wr: {total_win_rate/total_count}%, {player_stats[player_name]['win_rate']}%, delta: {total_win_rate/total_count - player_stats[player_name]['win_rate']}%")

adostraa's teammates vs self wr: 52.26627849023896%, 58.333333333333336%, delta: -6.067054843094375%
nigeriacoc's teammates vs self wr: 52.81631086649413%, 52.54237288135594%, delta: 0.2739379851381898%
TylerToTheT212's teammates vs self wr: 50.95676566961335%, 67.85714285714286%, delta: -16.900377187529514%
IbelieveMeYou's teammates vs self wr: 50.25657699521454%, 48.148148148148145%, delta: 2.108428847066392%
Ashalo1's teammates vs self wr: 52.22526322003085%, 39.58333333333333%, delta: 12.641929886697518%
ihyash's teammates vs self wr: 53.164495012619994%, 53.84615384615385%, delta: -0.6816588335338523%
DonutDude17's teammates vs self wr: 49.5373950101145%, 63.1578947368421%, delta: -13.620499726727601%
GasterBat's teammates vs self wr: 54.733751262599746%, 68.75%, delta: -14.016248737400254%
Bryant's teammates vs self wr: 51.70415964427321%, 66.66666666666666%, delta: -14.962507022393446%
uWumnati's teammates vs self wr: 52.46354663804703%, 33.33333333333333%, delta: 19.13021330471

In [23]:
champ = "Zac"

# Initialize player stats dictionary
zplayer_stats = defaultdict(lambda: {
    'wins': 0,
    'losses': 0,
    'kills': 0,
    'deaths': 0,
    'assists': 0,
    'matches': 0,
    'total_kills': 0,
    'penta_kills': 0,
    'games_played': 0
})

# Parse and extract data from each JSON file
for file_url in json_files_urls:
    match = fetch_json_from_url(file_url)
    if match:
        if 'participants' not in match:
            print(f"No participants data in file: {file_url}")
            continue
        participants = match['participants']
        # Group players by team
        teams = defaultdict(list)
        for player in participants:
            if 'NAME' not in player or 'TEAM' not in player:
                print(f"Incomplete player data in file: {file_url}")
                continue
            primary_name = get_primary_name(player['NAME'])
            team_id = player['TEAM']
            teams[team_id].append(primary_name)
        
        # Update player stats and teammate counts
        for player in participants:
            if 'NAME' not in player or 'TEAM' not in player:
                continue
            if player.get('SKIN') == champ:     
                player_name = get_primary_name(player['NAME'])  # Use primary name
                zplayer_stats[player_name]['matches'] += 1
                zplayer_stats[player_name]['kills'] += int(player.get('CHAMPIONS_KILLED', 0))
                zplayer_stats[player_name]['assists'] += int(player.get('ASSISTS', 0))
                zplayer_stats[player_name]['deaths'] += int(player.get('NUM_DEATHS', 0))
                zplayer_stats[player_name]['total_kills'] += int(player.get('CHAMPIONS_KILLED', 0))
                zplayer_stats[player_name]['penta_kills'] += int(player.get('PENTA_KILLS', 0))
                zplayer_stats[player_name]['games_played'] += 1

            if player.get('SKIN') == champ:
            # Check win/loss
                if player.get('WIN') == 'Win':
                    zplayer_stats[player_name]['wins'] += 1
                else:
                    zplayer_stats[player_name]['losses'] += 1
            
            # Update teammate counts for players on the same team
            team_id = player['TEAM']
            teammates = teams[team_id]
            for teammate in teammates:
                if teammate != player_name:  # Exclude the player themselves
                    teammate_counts[player_name][teammate] += 1

if not zplayer_stats:
    print("No player data found.")
else:
    # Compute win rate and average kills per match
    for player_name, stats in zplayer_stats.items():
        stats['win_rate'] = stats['wins'] / stats['matches'] * 100
        stats['avg_kills_per_match'] = stats['kills'] / stats['matches']
        
zsorted_players = sorted(zplayer_stats.items(), key=lambda x: x[1]['matches'], reverse=True)
# Print statistics for each player
print(f"Champion: {champ}")
for player_name, stats in zsorted_players:
    print(f"Player: {player_name}")
    print(f"  Win Rate: {stats['win_rate']:.2f}%, {stats['wins']}")
    print(f"  Games Played: {stats['games_played']}")
    print(f"  Average Kills per Match: {stats['avg_kills_per_match']:.2f}")
    avg_kda = (stats['kills'] + stats['assists']) / (stats['deaths'] if stats['deaths'] != 0 else 1)
    avg_kda = round(avg_kda, 2)
    print(f"  Average KDA: {avg_kda}")

Champion: Zac
Player: Bryant
  Win Rate: 33.33%, 3
  Games Played: 9
  Average Kills per Match: 2.44
  Average KDA: 2.7
Player: adostraa
  Win Rate: 66.67%, 2
  Games Played: 3
  Average Kills per Match: 4.00
  Average KDA: 3.58
Player: DonutDude17
  Win Rate: 100.00%, 1
  Games Played: 1
  Average Kills per Match: 3.00
  Average KDA: 15.0
